In [3]:
from textblob import TextBlob
import tweepy
import sys
import numpy as np


In [4]:

#Ham[0] 2007-09, 2010-2013, 2014-2016,2017-2020, 2021
#Seb[1] 2008-09, 2010-2013,2014-2018,2019-2020,2021
#Verstappen[2] 2015-2016,2016-2018,2019-2020, 2021 #10000 tweets per year

In [5]:
twkeys = open(r"twitterkeys.txt").read().splitlines() #waiting till twitter apikey is approved
tkey = twkeys[0]
tkeysecret = twkeys[1]
taccesstoken = twkeys[2]
taccesstokensecret = twkeys[3]


In [6]:
authhandler = tweepy.OAuthHandler(consumer_key=tkey,consumer_secret=tkeysecret)
authhandler.set_access_token(taccesstoken,taccesstokensecret)
api = tweepy.API(authhandler)

In [7]:
topic = np.array(['Lewis Hamilton','Sebastian Vettel','Max Verstappen'])
numtweets = 100000
tweets = []
for i in range(len(topic)):
    temp = tweepy.Cursor(api.search, q = topic[i], lang = 'en').items(numtweets) #textblob works best on english
    tweets.append(temp)
tweets = np.array(tweets)

In [8]:
#Lewis is index 0, Seb is index 1, Max is index 2
#positives = [] 
#negatives = []
#neutral = []
#polarities = []

for j in range(len(topic)):
    temppositive = 0
    tempnegative = 0
    tempneutral = 0
    temppolarity = 0
    for i in tweets[j]:
        cleaned  =  i.text.replace('RT','')
        if (cleaned.startswith(' @')):
            pos = cleaned.index(':')
            cleaned = cleaned[pos+2:]
        if (cleaned.startswith('@')):
            pos = cleaned.index(' ')
            cleaned = cleaned[pos+2:]
        
    
        currBlobToAnalyse = TextBlob(cleaned)
        currpolarity = currBlobToAnalyse.polarity
        if (currpolarity > 0.00):
            temppositive += 1
        elif (currpolarity < 0.00):
            tempnegative += 1
        elif (currpolarity == 0.00):
            tempneutral += 1
        temppolarity += currpolarity
    positives.append(temppositive)
    negatives.append(tempnegative)
    neutral.append(tempneutral)
    polarities.append(temppolarities)

TweepError: Twitter error response: status code = 401